In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate

# ML Predictive Modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

In [2]:
csv_f = '/Users/charisameeker/Documents/Data/Iowa Liquor/iowa_liquor_clean2.csv'
df1 = pd.read_csv(csv_f,
        parse_dates=['date'],
        dtype = {'year':np.int16, 'month':np.int8, 'day':np.int8,
        'store_number':np.int16, 'county_number':np.int8, 'item_number':np.int32,
        'pack':np.int16, 'bottle_vol':np.int64, 'bottle_cost':np.float32,
        'state_bottle_retail':np.float32, 'bottles_sold':np.int16, 'sale':np.float32,
        'vol_sold':np.float32, 'bottles_sold':np.int16,
        'category':np.int32, 'store_subnumber':np.int16, 'zip_code':np.int16,
        'vendor_number':np.int16
        })

In [5]:
df1.columns

Index(['year', 'month', 'day', 'store_number', 'city', 'zip_code',
       'county_number', 'category', 'vendor_number', 'item_number', 'pack',
       'bottle_vol', 'bottle_cost', 'state_bottle_retail', 'bottles_sold',
       'sale', 'fd', 'vol_sold', 'store_subnumber'],
      dtype='object')

In [ ]:
df1.drop(['Unnamed: 0', 'date', 'address', 'store_name', 'item_description', 'store_subname', \
        'county', 'category_name', 'vendor_name'], axis=1, inplace=True)

In [6]:
df1.zip_code = df1.zip_code.abs()

In [8]:
df1.city = df1.city.astype('category')
df1.zip_code = df1.zip_code.astype('category')

In [12]:
df1.city = df1.city.cat.add_categories('NaN')
df1.city.fillna('NaN', inplace=True)

In [14]:
df = df1.query('year == 2016')

In [16]:
file_pkl = '/Users/charisameeker/Documents/Data/iowa_liquor_clean4.cats.pkl'
df.to_pickle(file_pkl)

# Define Arrays

## Full dataset

In [17]:
y = df['fd'].copy().values # fd is the result of applying boxcox on the sale column (fitted data)
X = df.copy().drop(['sale', 'fd'], axis=1)
X_col = X.copy().columns

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [ ]:
# file_csv = '/Users/charisameeker/Desktop/iowa_liquor_2016_10k.csv'
# df.query('year == 2016').head(10000).to_csv(file_csv, index=False)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2279879 entries, 8428101 to 10707979
Data columns (total 19 columns):
 #   Column               Dtype   
---  ------               -----   
 0   year                 int16   
 1   month                int8    
 2   day                  int8    
 3   store_number         int16   
 4   city                 category
 5   zip_code             category
 6   county_number        int8    
 7   category             int32   
 8   vendor_number        int16   
 9   item_number          int32   
 10  pack                 int16   
 11  bottle_vol           int64   
 12  bottle_cost          float32 
 13  state_bottle_retail  float32 
 14  bottles_sold         int16   
 15  sale                 float32 
 16  fd                   float64 
 17  vol_sold             float32 
 18  store_subnumber      int16   
dtypes: category(2), float32(4), float64(1), int16(6), int32(2), int64(1), int8(3)
memory usage: 145.7 MB


# Plausible Regressors (without Parameter Tuning)

In [ ]:
regression_models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    LinearSVR(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    xgb.XGBRegressor()
]

for regression_model in regression_models:
    loop_pipe = make_pipeline(regression_model)
    loop_pipe.fit(X_train, y_train)   
    print(f'{regression_model} \n\
    model score: {loop_pipe.score(X_test03, y_test03):.4f}')

In [23]:
# pipeline for categorical features
categorical_features = ["city", "zip_code"]
# store_name, item_description, store_subname
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)

#incorporating the random forest estimator
pipeline = make_pipeline(categorical_transformer, RandomForestRegressor())

# Parameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'randomforestregressor__n_estimators': n_estimators,
               'randomforestregressor__max_features': max_features,
               'randomforestregressor__max_depth': max_depth,
               'randomforestregressor__min_samples_split': min_samples_split,
               'randomforestregressor__min_samples_leaf': min_samples_leaf,
               'randomforestregressor__bootstrap': bootstrap}
print(random_grid)

cv_rf = RandomizedSearchCV(pipeline, random_grid, cv=3)

{'randomforestregressor__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'randomforestregressor__max_features': ['auto', 'sqrt'], 'randomforestregressor__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'randomforestregressor__min_samples_split': [2, 5, 10], 'randomforestregressor__min_samples_leaf': [1, 2, 4], 'randomforestregressor__bootstrap': [True, False]}


In [ ]:
cv_rf.fit(X_train, y_train)

In [ ]:
print(f'Best parameters: {cv_rf.best_params_} \n\n\
Training accuracy score from tuned model: {cv_rf.best_score_*100:.2f}%')

In [ ]:
rf_y_pred = cv_rf.predict(X_test)

mse = mean_squared_error(y_test, rf_y_pred)
print(f'MSE: {mse:.2f}% \n\
RMSE: {(mse*(1/2.0)):.2} \n\
Score: {cv_rf.score(X_test, y_test)}')

In [ ]:
_ = plt.figure(figsize=(24, 12))
_ = x_ax = range(len(y_test[-200:]))
_ = plt.plot(x_ax, y_test[-200:], label="Occurences")
_ = plt.plot(x_ax, rf_y_pred[-200:], label="Predictions")
_ = plt.title('Random Forest Regressor for 2016: Occurences Vs. Predictions')
_ = plt.xlabel('Last 200 Instances')
_ = plt.ylabel('Fitted Sales (2016)')
_ = plt.legend()
_ = plt.show()

# Gradient Boosting Regressor

In [ ]:
# pipeline for categorical features
categorical_features = ["city", "zip_code"]
categorical_transformer = [("encoder", OneHotEncoder(handle_unknown='ignore'))]
categorical_transformer = Pipeline(categorical_transformer)




### DELETE
# divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("cat", categorical_transformer, categorical_features)])
### DELETE



#incorporating the random forest estimator
pipeline = make_pipeline(categorical_transformer, GradientBoostingRegressor())

# grid search parameters
params = {'gradientboostingregressor__learning_rate': [.1, .3, .5, .7, .9],
          'gradientboostingregressor__n_estimators': [50, 80, 100, 120, 140, 180, 250],
          'gradientboostingregressor__subsample': [.6, .8, 1, 1.2, 1.4, 1.9],
          'gradientboostingregressor__min_samples_split': [1, 2, 3, 4, 6]}

cv_gb = RandomizedSearchCV(pipeline, params, cv=3)

In [ ]:
cv_gb.fit(X_train, y_train)

In [ ]:
print(f'Best parameters: {cv_gb.best_params_} \n\n\
Training accuracy score from tuned model: {cv_gb.best_score_*100:.2f}%')

In [ ]:
gb_y_pred = cvcv_gb_rf.predict(X_test)

mse = mean_squared_error(y_test, gb_y_pred)
print(f'MSE: {mse:.2f}% \n\
RMSE: {(mse*(1/2.0)):.2} \n\
Score: {cv_gb.score(X_test, y_test)}')

In [ ]:
_ = plt.figure(figsize=(24, 12))
_ = x_ax = range(len(y_test[-200:]))
_ = plt.plot(x_ax, y_test[-200:], label="Occurences")
_ = plt.plot(x_ax, gb_y_pred[-200:], label="Predictions")
_ = plt.title('Gradient Boosting Regressor for 2016: Occurences Vs. Predictions')
_ = plt.xlabel('Last 200 Instances')
_ = plt.ylabel('Fitted Sales (2016)')
_ = plt.legend()
_ = plt.show()

# XGBoost Regressor

In [ ]:
parameters = {'xgbregressor__nthread':[4], #when use hyperthread, xgboost may become slower
              'xgbregressor__objective':['reg:linear'],
              'xgbregressor__learning_rate': [.03, 0.05, .07], #so called `eta` value
              'xgbregressor__max_depth': [5, 6, 7],
              'xgbregressor__min_child_weight': [4],
              'xgbregressor__silent': [1],
              'xgbregressor__subsample': [0.7],
              'xgbregressor__colsample_bytree': [0.7],
              'xgbregressor__n_estimators': [500]}

cv_xgb = RandomizedSearchCV(xgb.XGBRegressor(), parameters, cv=3)

In [ ]:
cv_xgb.fit(X_train, y_train)

In [ ]:
print(f'Best parameters: {cv_xgb.best_params_} \n\n\
Training accuracy score from tuned model: {cv_xgb.best_score_*100:.2f}%')

# 1. Linear Regression

In [ ]:
lin_reg1 = LinearRegression()
lin_reg1.fit(X_train, y_train)
y_pred = lin_reg1.predict(X_test)

In [ ]:
lin_reg1_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Linear Regression Score: {lin_reg1.score(X_test, y_test)} \n\n\
Mean Absolute Error: {metrics.mean_absolute_error(y_test, y_pred)} \n\n\
RMSE: {lin_reg1_rmse} \n\n\
Linear Regression Score: {lin_reg1.score(X_test, y_test)} \n\n\
Intercept: {lin_reg1.intercept_}')

In [ ]:
lin_reg1_df_y_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
lin_reg1_df_y_pred1 = df_y_pred.sample(25)

_ = lin_reg1_df_y_pred1.plot(kind='bar',figsize=(10,8))
_ = plt.title('2003: Occurences Vs. Predictions')
_ = plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
_ = plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
_ = plt.xlabel('Sample Index Numbers (25 Total)')
_ = plt.ylabel('Grade Median Scores')
_ = plt.xticks(rotation=80)
_ = plt.show()

In [ ]:
lin_reg2 = LinearRegression()

# 3-Fold Cross Validation
lin_reg2_cvscores_3 = cross_val_score(lin_reg2, X, y, cv=3)
print(f'Mean of 3 CV: {np.mean(lin_reg2_cvscores_3)}')

lin_reg2_cv_scores_5 = cross_val_score(lin_reg2, X, y, cv=5)
print(f'Mean of 5 CV: {(np.mean(lin_reg2_cv_scores_5))}')

# 10 Fold
lin_reg2_cvscores_10 = cross_val_score(lin_reg2reg2, X, y, cv=10)
print(f'Mean of 10 CV: { np.mean(lin_reg2_cvscores_10)}')

# 2. Ridge Regression and Alpha Optimization 

In [ ]:
ridge = Ridge()
ridge_parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X, y)

In [ ]:
print(f'Best Parameters: {ridge_regressor.best_params_} \n\
      Best Score: {ridge_regressor.best_score_} \n\
     Best Index: {ridge_regressor.best_index_} \n\
     Scorer: {ridge_regressor.scorer_} \n\
     Refit Time (Seconds): {ridge_regressor.refit_time_}')

In [ ]:
lasso1 = Lasso(alpha=.4)
lasso1_regressor = GridSearchCV(lasso1, parameters, scoring='neg_mean_squared_error', cv=3)
lasso1_regressor.fit(X, y)

In [ ]:
print(f'Best Parameters: {lasso_regressor.best_params_} \n\
     Best Score: {lasso_regressor.best_score_} \n\
     Best Index: {lasso_regressor.best_index_} \n\
     Scorer: {lasso_regressor.scorer_} \n\
     Refit Time (Seconds): {lasso_regressor.refit_time_}')

# 3. Lasso

In [ ]:
lasso2 = Lasso(alpha = .299, normalize = True)
lasso2.fit(X_train, y_train)

In [ ]:
print(f'Best Parameters: {lasso_regressor.best_params_} \n\
     Best Score: {lasso_regressor.best_score_} \n\
     Best Index: {lasso_regressor.best_index_} \n\
     Scorer: {lasso_regressor.scorer_} \n\
     Refit Time (Seconds): {lasso_regressor.refit_time_} \n\
     Score: {lasso.score(X_test, y_test)}')

# 4. Elastic Net

In [ ]:
# Create the hyperparameter grid
l1_space = np.linspace(0, 1, 30)
l1_space_param_grid = {'l1_ratio': l1_space}

# Instantiate the ElasticNet regressor: elastic_net
elastic_net = ElasticNet()

# Setup the GridSearchCV object: gm_cv
elastic_net_gm_cv = GridSearchCV(elastic_net, param_grid, cv=5)

# Fit it to the training data
elastic_net_gm_cv.fit(X_train, y_train)

# Predict on the test set and compute metrics
elastic_net_y_pred = gm_cv.predict(X_test)
elastic_net_r2 = gm_cv.score(X_test, y_test)
elastic_net_mse = mean_squared_error(y_test, elastic_net_y_pred)
print(f'Tuned ElasticNet l1 ratio: {gm_cv.best_params_} \n\
Tuned ElasticNet R squared: {elastic_net_r2} \n\
Tuned ElasticNet MSE: {elastic_net_mse}')

# 5. Stacked Regressor

In [ ]:
stacked_reg_estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=42))]

stacked_reg = StackingRegressor(estimators = stacked_reg_estimators,
        final_estimator = RandomForestRegressor(n_estimators = 10,
                                          random_state = 42))

stacked_reg.fit(X_train, y_train).score(X_test, y_test)

# 6. Random Forest Regressor

In [ ]:
rf_regr = RandomForestRegressor()
rf_regr.fit(X_train, y_train)

print(f'Regression Prediction: {rf_regr.predict([range(len(X.columns))])} \n\
Regression Score: {rf_regr.score(X_test, y_test)} \n\
Features: {rf_regr.n_features_}')

In [ ]:
rf_regr_df_y_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
rf_regr_df_y_pred = rf_regr_df_y_pred.sample(25)

_ = df_y_pred1.plot(kind='bar',figsize=(10,8))
_ = plt.title('2003 No Derivatives Random Forest Regressor: Occurences Vs. Predictions')
_ = plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
_ = plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
_ = plt.xlabel('Sample Index Numbers (25 Total)')
_ = plt.ylabel('Grade Median Scores')
_ = plt.xticks(rotation=80)
_ = plt.show()

# 7. Gradient Boosting Regressor

In [ ]:
gbr_reg = GradientBoostingRegressor()
gbr_reg.fit(X_train, y_train)
gbr_reg.predict(X_test)
print(f'Gradient Boosting Regressor score: {gbr_reg.score(X_test, y_test)}')

In [ ]:
gbr_reg_df_y_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
gbr_reg_df_y_pred1 = gbr_reg_df_y_pred.sample(25)

_ = gbr_reg_df_y_pred1.plot(kind='bar',figsize=(10,8))
_ = plt.title('2003 No Derivatives Gradient Boosting Regressor: Occurences Vs. Predictions')
_ = plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
_ = plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
_ = plt.xlabel('Sample Index Numbers (25 Total)')
_ = plt.ylabel('Grade Median Scores')
_ = plt.xticks(rotation=80)
_ = plt.show()

# 8. XGBoost

In [ ]:
xgb_dmatrix = xgb.DMatrix(data = X, label = y)

xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.45, learning_rate = 0.4,
                max_depth = 12, alpha = 6, n_estimators = 300)

xgb_reg.fit(X_train, y_train)
xgb_reg_y_pred = xgb_reg.predict(X_test)

xgb_reg_mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {round(xgb_reg_mse,2)} \n\
RMSE: {xgb_reg_mse*(1/2.0)} \n\
XGB score: {xgb_reg.score(X_test, y_test)}')

# LEAPS
print("MSE: %.2f" % xgb_reg_mse)
print("RMSE: %.2f" % (xgb_reg_mse*(1/2.0)))
xgb_reg.score(X_test, y_test)

## Visualize Accuracy

In [ ]:
xgb_reg_df_y_pred = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
xgb_reg_df_y_pred1 = xgb_reg_df_y_pred.sample(25)

_ = xgb_reg_df_y_pred1.plot(kind='bar',figsize=(10,8))
_ = plt.title('XGBoost: Occurences Vs. Predictions')
_ = plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
_ = plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
_ = plt.xlabel('Sample Index Numbers (25 Total)')
_ = plt.ylabel('Sales')
_ = plt.xticks(rotation=80)
_ = plt.show()

In [ ]:
_ = plt.figure(figsize=(24, 12))
_ = x_ax = range(len(y_test[-200:]))
_ = plt.plot(x_ax, y_test[-200:], label="Occurences")
_ = plt.plot(x_ax, xgb_reg_y_pred[-200:], label="Predictions")
_ = plt.title('XGBoost: Occurences Vs. Predictions')
_ = plt.xlabel('Last 200 Instances')
_ = plt.ylabel('Sales')
_ = plt.legend()
_ = plt.show()

# Pipeline

In [ ]:

from sklearn.ensemble import AdaBoostClassifier

numeric_features = ["avg_dist", "avg_rating_by_driver", "avg_rating_of_driver", \
                    "avg_surge", "surge_pct", "trips_in_first_30_days", "weekday_pct"]
numeric_transformer = [("scaler", StandardScaler())]
numeric_transformer = Pipeline(numeric_transformer)

#pipeline for categorical features
categorical_features = ["city", "phone", "ultimate_black_user", "active"]
categorical_transformer = [("encoder", OneHotEncoder())]
categorical_transformer = Pipeline(categorical_transformer)

#divernce of the pipelines
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_features), \
                  ("cat", categorical_transformer, categorical_features)])

#incorporating the random forest estimator
pipeline = Pipeline(steps=[("pre", preprocessor), \
                           ("ab", AdaBoostClassifier(random_state = 42))])

#grid search parameters
params = {"ab__n_estimators" : [75, 100, 150],
          "ab__learning_rate" : [.25, .5, .75, 1]}

#fitting the pipeline to the grid search
cv2 = GridSearchCV(pipeline, param_grid=params, cv=3)
cv2.fit(X_train, y_train)

print(cv2)

In [ ]:
from sklearn import metrics

y_pred = cv2.predict(X_test)

test_accuracy = metrics.accuracy_score(y_test, y_pred)
test_f1_score = metrics.f1_score(y_test, y_pred)

print(f"Model accuracy: {test_accuracy*100:.2f}%")
print(f"Model F1 Score: {test_f1_score:.2f}")

In [ ]:
#ada
pipeline.fit(X_train, y_train)

fe2 = pipeline.named_steps['ab'].feature_importances_

print(fe2)

In [ ]:
#replicating numeric_transformer of our pipeline
num_cols = df1[numeric_features].columns
num_cols = num_cols.tolist()

#replicating categorical_transformer of our pipeline
cat_cols = pd.get_dummies(df1[categorical_features]).columns
cat_cols = cat_cols.tolist()

#the column names are added together
cols = num_cols + cat_cols

#the column names are finally tied to our fe list
feature_importance = zip(cols, fe2)
feature_importance = sorted(feature_importance, key=lambda x:x[1], reverse=True)

for i, j in feature_importance:
    print(f"Weight: {j:.3f} | Feature: {i}")